In [ ]:
# default_exp client.__init__

# NerdGraph Client

## Imports

Python modules

In [ ]:
# export
import json
import os
import re
import pathlib
from textwrap import dedent

Third-party modules

In [ ]:
# export
import dotenv
from requests import Session

Development modules

In [ ]:
# hide
from nbdev.showdoc import show_doc

## Classes and functions

In [ ]:
# export
def get_new_relic_user_key_from_env() -> str:
    env_file = pathlib.Path(".env")

    if env_file.exists():
        dotenv.load_dotenv(env_file)

    new_relic_user_key = os.environ.get("NEW_RELIC_USER_KEY", None)

    if new_relic_user_key is None:
        raise ValueError("Environment variable NEW_RELIC_USER_KEY is not set.")

    return new_relic_user_key

In [ ]:
show_doc(get_new_relic_user_key_from_env)

In [ ]:
# export
class NewRelicClient(Session):
    url: str = "https://api.newrelic.com/graphql"

    def __init__(self, *, new_relic_user_key: str):
        super().__init__()

        self.headers.update(
            {
                "Content-Type": "application/json",
                "API-Key": new_relic_user_key,
            }
        )

    def execute(
        self, query=None, query_kwargs=None, **kwargs
    ):  # pylint: disable=arguments-differ
        data = self._build_query(query, **(query_kwargs or {}))
        return self.post(self.url, data=data, **kwargs)

    @staticmethod
    def _build_query(query_string, **kwargs):
        return json.dumps({"query": dedent(query_string.strip()) % kwargs})

In [ ]:
show_doc(NewRelicClient)

In [ ]:
# export
class NewRelicApiClient(Session):
    url: str = "https://api.newrelic.com/v2/"

    def __init__(self, *, new_relic_user_key: str):
        super().__init__()

        self.headers.update(
            {
                "Content-Type": "application/json",
                "Api-Key": new_relic_user_key,
            }
        )

    def execute(self, parameters=None, apiFile: str | None = "alerts_conditions.json"):
        response = self.get(self.url + apiFile, params=parameters)

        if response.status_code == 200:
            return response.json()

        return None